In [36]:
import numpy as np
import glob
import os
import math

In [83]:
import os
import glob

def ajustar_args_n_samples(args, n_samples):
    n_threads = os.cpu_count()  # Número de threads fixado como o número de CPUs
    len_args = len(args)

    if len_args == 0:
        raise ValueError("A lista de argumentos (args) está vazia.")
    if n_threads == 0:
        raise ValueError("O número de threads (n_threads) não pode ser zero.")

    # Dividir os argumentos em múltiplos de n_threads e o restante
    num_multiplo_threads = (len_args // n_threads) * n_threads
    args_multiplo = args[:num_multiplo_threads]  # Parte múltipla de n_threads
    args_restantes = args[num_multiplo_threads:]  # Restante

    # Ajustar o número de samples para o grupo principal
    if num_multiplo_threads > 0:
        n_samples_main = n_samples * num_multiplo_threads // len_args
    else:
        n_samples_main = 0

    # Ajustar os argumentos restantes para um múltiplo de n_threads
    if len(args_restantes) > 0:
        fator_repeticao = -(-n_threads // len(args_restantes))  # Arredondar para cima
        args_restantes_expandidos = (args_restantes * fator_repeticao)[:n_threads]
        n_samples_restantes = n_samples * len(args_restantes) // len_args
    else:
        args_restantes_expandidos = []
        n_samples_restantes = 0

    return args_multiplo, n_samples_main, (args_restantes_expandidos, n_samples_restantes)


def gerar_script(filename, arguments, n_samples):
    with open(filename, "w") as f:
        f.write("#!/bin/bash\n\n")
        f.write(f"arguments=({' '.join(arguments)})\n")
        f.write(f"n_samples={n_samples}\n")
        f.write("x=0\n")
        f.write("while [ $x -lt $n_samples ]; do\n")
        f.write("\tparallel run_code ::: \"${arguments[@]}\"\n")
        f.write("\tx=$(( $x + 1 ))\n")
        f.write("done\n")

def multithread_pc(N, NumSamples):
    n_threads = os.cpu_count()  # Número de threads fixado
    filename1 = f"N_{N}_multithread_pc_main.sh"
    filename2 = f"N_{N}_multithread_pc_extra.sh"

    # Caminho para os arquivos JSON
    path_d = f"../../parms_pc_{N}"
    all_files = glob.glob(os.path.join(path_d, "*.json"))
    list_of_arguments = [os.path.basename(f) for f in all_files]

    # Ajustar argumentos e n_samples
    args_main, n_samples_main, extra = ajustar_args_n_samples(list_of_arguments, NumSamples)

    # Gerar o script principal
    if args_main:
        gerar_script("../" + filename1, args_main, n_samples_main)

    # Gerar o script extra, se necessário
    if extra[0]:  # Se existem argumentos restantes
        args_extra, n_samples_extra = extra
        gerar_script("../" + filename2, args_extra, n_samples_extra)



In [84]:
N= 160000
n_threads = 20
# Caminho para os arquivos JSON
path_d = f"../../parms_pc_{N}"
all_files = glob.glob(os.path.join(path_d, "*.json"))
list_of_arguments = [os.path.basename(f) for f in all_files]
A = ajustar_args_n_samples(list_of_arguments, 30)
A
#print(list_of_arguments)


([],
 0,
 (['N160000_a7.00_g2.00_d3.0.json',
   'N160000_a13.00_g2.00_d4.0.json',
   'N160000_a8.00_g2.00_d4.0.json',
   'N160000_a1.00_g2.00_d4.0.json',
   'N160000_a10.00_g2.00_d4.0.json',
   'N160000_a6.00_g2.00_d3.0.json',
   'N160000_a5.00_g2.00_d2.0.json',
   'N160000_a14.00_g2.00_d4.0.json',
   'N160000_a12.00_g2.00_d4.0.json',
   'N160000_a8.00_g2.00_d3.0.json',
   'N160000_a11.00_g2.00_d4.0.json',
   'N160000_a9.00_g2.00_d4.0.json',
   'N160000_a7.00_g2.00_d4.0.json',
   'N160000_a9.00_g2.00_d3.0.json',
   'N160000_a7.00_g2.00_d3.0.json',
   'N160000_a13.00_g2.00_d4.0.json',
   'N160000_a8.00_g2.00_d4.0.json',
   'N160000_a1.00_g2.00_d4.0.json',
   'N160000_a10.00_g2.00_d4.0.json',
   'N160000_a6.00_g2.00_d3.0.json'],
  30))

In [73]:
N = 160000
NumSamples = 30

multithread_pc(N, NumSamples)


In [52]:
N_a = os.cpu_count()
N = 160000
path_d = f"../../parms_pc_{N}"
all_files = glob.glob(os.path.join(path_d,"*.json"))
#N_t = len(all_files)
N_t = 14
N_div = (N_t / N_a) % N_t
frac, inte = math.modf(N_div)
print(frac, inte)


# list_of_arguments = [V[2] for V in os.walk(path_d)][0]
# list_of_arguments = str(list_of_arguments)
# list_of_arguments = list_of_arguments.replace(',', '')

# h = f"arguments=(" 
# i = list_of_arguments[1:-1] + ")\n\n"
# j = "x=0\n"

0.7 0.0
